In [1]:
from docling.document_converter import DocumentConverter

source = "data/$REQHDCW (2).pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
md_document = result.document.export_to_markdown()
teretovnica = "\n".join(md_document.split("## Teretovnica")[1:])
with open("current_teretovnica.md", "w") as f:
    f.write(teretovnica)
print(teretovnica)

/home/davor/miniconda3/envs/sasha_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Could not load the custom kernel for multi-scale deformable attention: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.
Could not load the custom kernel for multi-scale deformable attention: /home/davor/.cache/torch_extensions/py312_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/davor/.cache/torch_extensions/py312_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /home/davor/.cache/t



| Rbr. Sadržaj upisa                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                | Iznos         | Primjedba   |
|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [20]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

source = "data/$REQHDCW (2).pdf"  # document per local path or URL
converter = PdfConverter(
    artifact_dict=create_model_dict(),
)
rendered = converter(source)
text, _, images = text_from_rendered(rendered)

Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16
Loaded texify model s3://texify/2025_02_18 on device cuda with dtype torch.float16
Loaded recognition model s3://text_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded table recognition model s3://table_recognition/2025_02_18 on device cuda with dtype torch.float16
Loaded detection model s3://text_detection/2025_02_28 on device cuda with dtype torch.float16
Loaded detection model s3://inline_math_detection/2025_02_24 on device cuda with dtype torch.float16


Running OCR Error Detection: 100%|██████████| 3/3 [00:00<00:00, 154.13it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 3/3 [00:01<00:00,  2.10it/s]


In [21]:
print(text)

![](_page_0_Picture_1.jpeg)

#### **Općinski sud u Novom Zagrebu ZEMLJIŠNOKNJIŽNI ODJEL NOVI ZAGREB Stanje na dan: 21.02.2025. 14:28** Verificirani ZK uložak

### **Katastarska općina: 9996, BLATO NOVO Broj ZK uloška: 2827**

Broj zadnjeg dnevnika: Z-31837/2024 **ETAŽNO VLASNIŠTVO S ODREĐENIM OMJERIMA** Aktivne plombe:

## **IZVADAK IZ ZEMLJIŠNE KNJIGE**

| Rbr. | Broj<br>zemljišta<br>(kat.<br>čestice) | Oznaka zemljišta                                    | Površina |     |      |           |
|------|----------------------------------------|-----------------------------------------------------|----------|-----|------|-----------|
|      |                                        |                                                     | jutro    | čhv | m2   | Primjedba |
| 1.   | 1342/2                                 | STAMBENA ZGRADA BR. 17 I 17 A I DVORIŠTE, JARUŠČICA |          |     | 1569 |           |
|      |                                        | STAMBENA ZGRADA BR. 17 I 17 A   

In [22]:
teretovnica = text[text.index("# **C") :].replace("<br>", " ")  # "\n".join(text.split("**C**")[1:])

In [23]:
print(teretovnica)

# **C**

| Rbr. | Sadržaj upisa                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              | Iznos    

In [ ]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.tools import BaseTool, StructuredTool, tool


class Liability(BaseModel):
    liability_id: str = Field(description="the liability ID, starts with 'Z-'")
    liability_received_date: str = Field(
        description="date of receiving the liability, usually written after 'Zaprimljeno', in the format DD.MM.YYYY."
    )
    credit_issued_date: Optional[str] = Field(
        default=None,
        description="the date when the credit was issued, often the first date mentioned in the liability description, in the format DD.MM.YYYY.",
    )
    all_dates: list[str] = Field(
        default=[], description="all dates found in the liability description, in the format DD.MM.YYYY."
    )
    liability_amount: Optional[float] = Field(default=None, description="the monetary amount of the liability")
    creditor_info: Optional[str] = Field(
        default=None, description="the liability creditor, and their information if available (OIB, address, etc.)"
    )


class AllLiabilities(BaseModel):
    liabilities: list[Liability] = Field(description="a list of all liabilities")


@tool("liability-parse", args_schema=AllLiabilities)
def liability_parse(all_liabilities: AllLiabilities) -> str:
    """Pass the parsed information for all liabilities."""
    print(all_liabilities)


TERETOVNICA_PROMPT = """You are a proficient document parser. Your task is to parse the given document written in Croatian.
The document lists all the liabilities of an individual, each under its own number ("Rbr." in document). Multiple liabilities can be found under the same "Rbr."
You have to parse every liability available, in order of their appearance.
You must extract the following information for each liability:
    - liability_id: the liability ID (starts with Z-)
    - liability_received_date: date when the liability was received (usually mentioned after "Zaprimljeno")
    - credit_issued_date: the date when the credit was issued, usually the first date in the liability description after the liabilities_received_date
    - all_dates: all dates found in the liability description
    - liability_amount: the monetary amount of the liability
    - creditor_info: creditor information (their name, OIB and address if available), the creditor is often a bank (but can be other entities, even people), the creditor is most likely written in capital letters at the end of the liability listing

Each liability entry is formatted as following:
| Rbr. | Sadržaj upisa | Iznos | Primjedba |
|------|---------------|-------|----------|
| <liability_number> | Zaprimljeno <liabilities_received_date> pod brojem <liabilities_id>\n<liabilities_description> | <liabilities_amount> | <optional_note> |

<liabilities_description> is a long text that contains the credit_start_date and creditor_info, among other information.
<credit_start_date> is the date when the credit was issued, usually the first date mentioned in the liability description.
<creditor_info> is the creditor information, often written at the end of the liability description.

Format your output as a JSON.

If there are no liabilities listed, return an empty list."""

"""
For example:
INPUT:
| Rbr. | Sadržaj upisa                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | Iznos         | Primjedba                |
|------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|---------------|--------------------------|
| 24.1 | Zaprimljeno 25.05.2020.g. pod brojem Z-10024/2020                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 63.000,00 EUR | vezano uz B 19 (1.2)     |
|      | UKNJIŽBA, ZALOŽNO PRAVO, SPORAZUM O ZASNIVANJU ZALOŽNOG<br>PRAVA (HIPOTEKE) NA NEKRETNINI BROJ 9916829136-006 15.05.2020,<br>PUNOMOĆ OV-285/20 28.10.2019, radi osiguranja novčane tražbine u iznosu od<br>63.000,00 EUR u kunskoj protuvrijednosti po srednjem tečaju tečajne liste HNB<br>za 1 EUR važeće na dan korištenja kredita s načinom otplate u mjesečnim<br>anuitetima, uvećano za sve ugovorene redovne kamate, eventualne zakonske<br>zatezne kamate, koje su promjenjive te ostala sporedna potraživanja, te prema<br>ostalim uvjetima iz sporazuma i ugovora,<br>HPB D.D., OIB: 87939104217, JURIŠIĆEVA ULICA 4, 10000 ZAGREB                                                                                                                                                                                                                                                                                                                                                                                                       |               |                          |
|      | 25. Na suvlasnički dio: 3 (230/10000)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |               |                          |
| 25.1 | Zaprimljeno 12.06.2020.g. pod brojem Z-11597/2020                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  | 94.000,00 EUR | vezano uz B 3 (1.1)      |
|      | UKNJIŽBA, ZALOŽNO PRAVO, SPORAZUM O ZASNIVANJU ZALOŽNOG<br>PRAVA NA NEKRETNINAMA I PRAVU NA NEPOSREDNU OVRHU OD<br>25.05.2020. POTVRĐEN PO JAVNOM BILJEŽNIKU MAJI PERIS POD POSL.<br>BROJEM: OV-4968/2020 DANA 12.06.2020, u iznosu od 94.000,00 EUR<br>(slovima: devedesetčetiritisućeEUR) u kunskoj protuvrijednosti po srednjem<br>tečaju za EUR tečajne liste Hrvatske narodne banke s ugovorenim naknadama i<br>troškovima i kamatom koja je za razdoblje otplate kredita od 96 mjeseci fiksna i<br>iznosi 2,28 % (slovima: dvacijelihdvadesetosamposto), godišnje, a nakon isteka<br>navedenog razdoblja kamatna stopa je promjenjiva i definira se kao zbroj<br>referentne kamatne stope u visini šestomjesečnog NRS1 za EUR, promjenjivo i<br>fiksnog dijela koji iznosi 2,03 p.p. (dvacijelihnulatripostotnihpoena) godišnje, a<br>koja na dan citiranog Sporazuma iznosi 2,28 % (dvacijelihdvadesetosamposto),<br>godišnje, promjenjivo, za korist:<br>PRIVREDNA BANKA ZAGREB D.D., OIB: 02535697732, RADNIČKA CESTA<br>50, 10000 ZAGREB |               |                          |
|      | 26. Na suvlasnički dio: 23 (231/10000)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |               |                          |
| 26.1 | Zaprimljeno 27.09.2021.g. pod brojem Z-24996/2021<br>UKNJIŽBA, ZALOŽNO PRAVO, UGOVOR O STAMBENOM POTROŠAČKOM<br>KREDITU 23.09.2021, radi osiguranja novčane tražbine u iznosu od 70.000,00<br>EUR u kunskoj protuvrijednosti po srednjem tečaju za EUR tečajne liste HNB s<br>ugovorenim naknadama i troškovima i kamatom koja je za razdoblje otplate<br>kredita od 60 mjeseci fiksna i iznosi 3,18% godišnje, a nakon isteka navedenog<br>razdoblja kamatna stopa je promjenjiva i definira se kao zbroj referentne kamatne<br>stope u visini 6M NRS1 EUR, promjenjivo i fiksnog dijela koji iznosi 3,23 p.p.<br>godišnje, a koja na dan citiranog Ugovora iznosi 3,34% godišnje, promjenjivo i<br>svih ostalih uvjeta iz cit.Ugovora za korist:<br>PRIVREDNA BANKA ZAGREB D.D., OIB: 02535697732, RADNIČKA CESTA<br>50, 10000 ZAGREB                                                                                                                                                                                                            | 70.000,00 EUR | vezano uz B 23<br>(23.1) |
|      | 27. Na suvlasnički dio: 4 (227/10000)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              |               |                          |
| 27.1 | Zaprimljeno 24.02.2022.g. pod brojem Z-4956/2022                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   | 68.800,00 EUR | vezano uz B 4 (4.1)      |
|      | UKNJIŽBA, ZALOŽNO PRAVO, UGOVOR O KREDITU 24. veljače 2022, radi<br>osiguranja novčanog potraživanja u iznosu od 68.800,00 EUR (slovima:<br>šezdesetosamtisućaosamsto eura) u kunskoj protuvrijednosti obračunato po<br>srednjem tečaju HNB na dan isplate kredita, s pripadajućim kamatama u vrijeme i<br>na način utvrđen Ugovorom<br>ZAGREBAČKA BANKA D.D., OIB: 92963223473, TRG BANA JELAČIĆA 10,<br>10000 ZAGREB<br>28. Na suvlasnički dio: 41 (296/10000)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |               |                          |
OUTPUT:
{{
    "liabilities":
        [
        {{"liabilities_id": "Z-10024/2020", "liabilities_received_date": "25.05.2020", "credit_start_date": "15.05.2020", "liabilities_amount": 63000.0, "creditor_info": "HPB D.D., OIB: 87939104217, JURIŠIĆEVA ULICA 4, 10000 ZAGREB"}},
        {{"liabilities_id": "Z-11597/2020", "liabilities_received_date": "12.06.2020", "credit_start_date": "25.05.2020", "liabilities_amount": 94000.0, "creditor_info": "PRIVREDNA BANKA ZAGREB D.D., OIB: 02535697732, RADNIČKA CESTA 50, 10000 ZAGREB"}},
        {{"liabilities_id": "Z-24996/2021", "liabilities_received_date": "27.09.2021", "credit_start_date": "23.09.2021", "liabilities_amount": 70000.0, "creditor_info": "PRIVREDNA BANKA ZAGREB D.D., OIB: 02535697732, RADNIČKA CESTA 50, 10000 ZAGREB"}},
        {{"liabilities_id": "Z-4956/2022 ", "liabilities_received_date": "24.02.2022", "credit_start_date": "24.02.2022", "liabilities_amount": 68800.0, "creditor_info": "ZAGREBAČKA BANKA D.D., OIB: 92963223473, TRG BANA JELAČIĆA 10, 10000 ZAGREB 28"}},
    ]
}}
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", TERETOVNICA_PROMPT), ("user", "Here is the document:\n{teretovnica}")]
)

llm = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.0).with_structured_output(AllLiabilities)
output = llm.invoke({"teretovnica": teretovnica})

In [53]:
from datetime import datetime

for liability in output.liabilities:
    if liability.liability_amount is not None:
        if liability.credit_issued_date is None:
            parsed_dates = [datetime.strptime(date, "%d.%m.%Y.") for date in liability.all_dates]
            liability.credit_issued_date = min(parsed_dates).strftime("%d.%m.%Y.")
        print(liability.model_dump())

{'liability_id': 'Z-62094/10', 'liability_received_date': '27.12.2010.', 'credit_issued_date': '23.12.2010.', 'all_dates': ['27.12.2010.', '23.12.2010.'], 'liability_amount': 37440.0, 'creditor_info': 'RAIFFEISENBANK AUSTRIA D.D., OIB: 53056966535, ZAGREB, PETRINJSKA 59'}
{'liability_id': 'Z-40082/10', 'liability_received_date': '18.08.2010.', 'credit_issued_date': '12.08.2010.', 'all_dates': ['18.08.2010.', '12.08.2010.'], 'liability_amount': 41000.0, 'creditor_info': 'ZAGREBAČKA BANKA D.D., OIB: 92963223473, PAROMLINSKA BR. 2, ZAGREB'}
{'liability_id': 'Z-36003/10', 'liability_received_date': '19.07.2010.', 'credit_issued_date': '05.07.2010.', 'all_dates': ['19.07.2010.', '05.07.2010.'], 'liability_amount': 62024.24, 'creditor_info': 'ERSTE&STEIERMÄRKISCHE BANK D.D., OIB: 23057039320, JADRANSKI TRG 3A, 51000 RIJEKA'}
{'liability_id': 'Z-30489/10', 'liability_received_date': '15.06.2010.', 'credit_issued_date': '04.06.2010.', 'all_dates': ['15.06.2010.', '04.06.2010.'], 'liability_amo